02/22/2023_Shri_kumbhar

In [1]:
%%capture
# !pip install datasets==1.13.3
# !pip install transformers==4.11.3
!pip install git+https://github.com/huggingface/transformers

!pip install torchaudio
!pip install librosa
!pip install jiwer

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install git+https://github.com/huggingface/datasets.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/datasets.git to /tmp/pip-req-build-uscflvr0
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/datasets.git /tmp/pip-req-build-uscflvr0
  Resolved https://github.com/huggingface/datasets.git to commit a940972a9a38543b2066129dc6e7987e08dca082
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 17.7 MB/s eta 0:00:00
  Created wheel for datasets: filename=datasets-2.10.1.dev0-py3-none-any.whl size=469381 sha256=e604ec2908eaef7bad1ad38d6f4ec9e6e59537512496a48d2a2c6e78b8a88692
  Stored in directory: /tmp/pip-ephem-wheel-

In [4]:
import transformers
print(transformers.__version__)


4.27.0.dev0


In [5]:
import datasets
print(datasets.__version__)


2.10.1.dev0


In [6]:
import torch

In [7]:
from datasets import load_dataset, load_metric, Audio

common_voice_train = load_dataset("google/fleurs",'en_us',split='all')

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset fleurs downloaded and prepared to /root/.cache/huggingface/datasets/google___fleurs/en_us/2.0.0/af82dbec419a815084fa63ebd5d5a9f24a6e9acdf9887b9e3b8c6bbd64e0b7ac. Subsequent calls will reuse this data.


In [8]:
asr_data = common_voice_train.train_test_split(test_size=0.2)

In [9]:
asr_data

DatasetDict({
    train: Dataset({
        features: ['id', 'num_samples', 'path', 'audio', 'transcription', 'raw_transcription', 'gender', 'lang_id', 'language', 'lang_group_id'],
        num_rows: 2914
    })
    test: Dataset({
        features: ['id', 'num_samples', 'path', 'audio', 'transcription', 'raw_transcription', 'gender', 'lang_id', 'language', 'lang_group_id'],
        num_rows: 729
    })
})

In [10]:
asr_data = asr_data.remove_columns(['id', 'num_samples', 'path','raw_transcription', 'gender', 'lang_id', 'language', 'lang_group_id'])

In [11]:
asr_data

DatasetDict({
    train: Dataset({
        features: ['audio', 'transcription'],
        num_rows: 2914
    })
    test: Dataset({
        features: ['audio', 'transcription'],
        num_rows: 729
    })
})

In [12]:
asr_data['train'][0]

{'audio': {'path': 'train/478546009124379672.wav',
  'array': array([0.        , 0.        , 0.        , ..., 0.00228405, 0.00231534,
         0.00287175]),
  'sampling_rate': 16000},
 'transcription': 'machu picchu consist of three main structures namely intihuatana the temple of the sun and the room of the three windows'}

In [13]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

show_random_elements(asr_data["train"].remove_columns(["audio"]))


,transcription
0,huhne has resigned and he will be replaced in the cabinet by ed davey mp norman lamb mp is expected to take the business minister job davey is vacating
1,willingness of foreign governments to honour these documents is just as widely variable
2,the games kicked off at 10:00am with great weather and apart from mid morning drizzle which quickly cleared up it was a perfect day for 7's rugby
3,a doctor who worked at children's hospital of pittsburgh pennsylvania will be charged with aggravated murder after her mother was found dead in the trunk of her car wednesday authorities in ohio say
4,many people don't think about them as dinosaurs because they have feathers and can fly
5,the irish government is stressing the urgency of parliamentary legislation to rectify the situation
6,he went on to say this case is serious rest assured that our system is working as well as it should
7,by early today winds were around 83 km/h and it was expect to keep weakening
8,the most readily accessible plant resources would have been the proteins accessible in leaves and legumes but these are hard for primates like us to digest unless they are cooked
9,the believer seeks a direct experience intuition or insight into divine reality/the deity or dieties


In [14]:
import re
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"]$\u200b%'

def remove_special_characters(batch):
    batch["transcription"] = re.sub(chars_to_ignore_regex, '', batch["transcription"]).lower()
    return batch

asr_data = asr_data.map(remove_special_characters)


Map:   0%|          | 0/2914 [00:00<?, ? examples/s]

Map:   0%|          | 0/729 [00:00<?, ? examples/s]

In [15]:
show_random_elements(asr_data["train"].remove_columns(["audio"]))


,transcription
0,towards the end of the middle ages western europe began to develop their own style one of the biggest developments of the time as a result of the crusades people began to use buttons to fasten clothing
1,this is the first time a conviction has been gained using the legislation enacted in 2003 to curb bulk e-mail aka spam from unsolicited distribution into users mailboxes
2,it is not known at this time what charges will be laid or what led authorities to the boy but juvenile proceedings have begun in federal court
3,the eathquake struck mariana at 07:19 a.m. local time 09:19 p.m. gmt friday
4,a doctor who worked at children's hospital of pittsburgh pennsylvania will be charged with aggravated murder after her mother was found dead in the trunk of her car wednesday authorities in ohio say
5,the high middle ages were preceded by the early middle ages and followed by the late middle ages which by convention ends around 1500
6,former house speaker newt gingrich texas governor rick perry and congresswoman michele bachmann finished in fourth fifth and sixth place respectively
7,women should realize that cultural differences may result in what they would consider harassment and it is not uncommon to be followed grabbed by the arm etc
8,cochamó valley - chile's premier climbing destination known as the yosemite of south america with a variety of granite big walls and crags
9,re-entry shock comes on sooner than culture shock there's less of a honeymoon phase lasts longer and can be more severe


In [16]:
asr_data

DatasetDict({
    train: Dataset({
        features: ['audio', 'transcription'],
        num_rows: 2914
    })
    test: Dataset({
        features: ['audio', 'transcription'],
        num_rows: 729
    })
})

In [ ]:
def extract_all_chars(batch):
  all_text = " ".join(batch["transcription"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}


In [ ]:
# vocabs = asr_data.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=asr_data.column_names["train"])

In [17]:
# # Define a list of lowercase letters from a to z
# letters = [chr(i) for i in range(ord('a'), ord('z') + 1)]

# # Create a dictionary that maps each letter to its index in the list
# vocab_dict = {letter: index for index, letter in enumerate(letters)}
# vocab_dict[' '] = 26

vocab_dict = {    
     ' ': 21,
     "'": 13,
     'a': 24,
     'b': 17,
     'c': 25,
     'd': 2,
     'e': 9,
     'f': 14,
     'g': 22,
     'h': 8,
     'i': 4,
     'j': 18,
     'k': 5,
     'l': 16,
     'm': 6,
     'n': 7,
     'o': 10,
     'p': 19,
     'q': 3,
     'r': 20,
     's': 11,
     '.': 0,
     'u': 26,
     'v': 27,
     'w': 1,
     'x': 23,
     'y': 15,
     'z': 12,
     't': 28
}


In [18]:
sorted(vocab_dict.values())

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28]

In [ ]:
# # vocab_list = list(set(vocabs["train"]["vocab"][0]) | set(vocabs["test"]["vocab"][0]))

# for i in vocab_dict.keys():
#   vocab_dict[i]+=1
# vocab_dict

In [19]:
vocab_dict

{' ': 21,
 "'": 13,
 'a': 24,
 'b': 17,
 'c': 25,
 'd': 2,
 'e': 9,
 'f': 14,
 'g': 22,
 'h': 8,
 'i': 4,
 'j': 18,
 'k': 5,
 'l': 16,
 'm': 6,
 'n': 7,
 'o': 10,
 'p': 19,
 'q': 3,
 'r': 20,
 's': 11,
 '.': 0,
 'u': 26,
 'v': 27,
 'w': 1,
 'x': 23,
 'y': 15,
 'z': 12,
 't': 28}

In [20]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]


In [42]:
asr_data

DatasetDict({
    train: Dataset({
        features: ['audio', 'transcription'],
        num_rows: 2914
    })
    test: Dataset({
        features: ['audio', 'transcription'],
        num_rows: 729
    })
})

In [21]:
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
print(len(vocab_dict))


31


In [22]:
import json
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)


In [23]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer("/content/vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")


In [24]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=False)


In [25]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)


In [26]:
import IPython.display as ipd
import numpy as np
import random

rand_int = random.randint(0, len(asr_data["train"]))

print(asr_data["train"][rand_int]["transcription"])
ipd.Audio(data=np.asarray(asr_data["train"][rand_int]["audio"]["array"]), autoplay=True, rate=16000)


the man suspected of detonating the bomb was detained after sustaining injuries from the blast


In [27]:
rand_int = random.randint(0, len(asr_data["train"]))

print("Target text:", asr_data["train"][rand_int]["transcription"])
print("Input array shape:", np.asarray(asr_data["train"][rand_int]["audio"]["array"]).shape)
print("Sampling rate:", asr_data["train"][rand_int]["audio"]["sampling_rate"])


Target text: he was reportedly aged in his 20s. in a statement bieber said [w]hile i was not present nor directly involved with this tragic accident my thoughts and prayers are with the family of the victim.
Input array shape: (244800,)
Sampling rate: 16000


In [43]:
def prepare_dataset(batch):
    audio = batch["audio"]

    # batched output is "un-batched" to ensure mapping is correct
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["transcription"]).input_ids
    return batch


In [44]:
asr_data

DatasetDict({
    train: Dataset({
        features: ['audio', 'transcription'],
        num_rows: 2914
    })
    test: Dataset({
        features: ['audio', 'transcription'],
        num_rows: 729
    })
})

In [ ]:
# def prepare_dataset(batch):
#     audio = batch["audio"]

#     # batched output is "un-batched" to ensure mapping is correct
#     batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]

#     with processor.as_target_processor():
#         batch["labels"] = processor(batch["transcription"]).input_ids

#     # Find the maximum batch shape
#     max_batch_shape = batch["input_values"].shape
#     for key, value in batch["input_values"].items():
#         if isinstance(value, np.ndarray) and len(value.shape) > 0:
#             if value.shape[0] > max_batch_shape[0]:
#                 max_batch_shape = (value.shape[0],)

#     print("Max input length across the batch:", max_batch_shape[0])

#     # Find the maximum label length
#     max_label_length = batch["labels"].shape[1]
#     for key, value in batch["labels"].items():
#         if isinstance(value, np.ndarray) and value.shape[1] > max_label_length:
#             max_label_length = value.shape[1]

#     print("Max label length across the batch:", max_label_length)

#     return batch


In [45]:
asr_data = asr_data.map(prepare_dataset, remove_columns=asr_data.column_names["train"], num_proc=4)


Map (num_proc=4):   0%|          | 0/2914 [00:00<?, ? examples/s]

/usr/local/lib/python3.8/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__`

Map (num_proc=4):   0%|          | 0/729 [00:00<?, ? examples/s]

/usr/local/lib/python3.8/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__`

In [ ]:
max_val_input = 0
max_val_label = 0
for i in range(len(asr_data['train'])):
  if len(asr_data['train'][i]['input_values'])>max_val_input:
    max_val_input = len(asr_data['train'][i]['input_values'])
  if len(asr_data['train'][i]['labels'])>max_val_label:
    max_val_label = len(asr_data['train'][i]['labels'])
print(max_val_input)
print(max_val_label)
# len(asr_data['train'][0]['input_values'])

677760
287


In [ ]:
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [29]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch


In [30]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding='longest')


In [31]:
wer_metric = load_metric("wer")


<ipython-input-31-605de035dfd7>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  wer_metric = load_metric("wer")


In [32]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}


In [33]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-base", 
    ctc_loss_reduction="mean", 
    pad_token_id=processor.tokenizer.pad_token_id,
)


/usr/local/lib/python3.8/dist-packages/transformers/configuration_utils.py:379: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


Some weights of the model checkpoint at facebook/wav2vec2-base were not used when initializing Wav2Vec2ForCTC: ['project_q.weight', 'project_hid.weight', 'project_hid.bias', 'quantizer.codevectors', 'project_q.bias', 'quantizer.weight_proj.weight', 'quantizer.weight_proj.bias']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['lm_head.bias', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predicti

In [ ]:
model.freeze_feature_extractor()


In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir='/content/drive/MyDrive/wave2vecmodels',
  group_by_length=True,
  per_device_train_batch_size=8,
  evaluation_strategy="steps",
  num_train_epochs=50,
  fp16=True,
  gradient_checkpointing=True, 
  save_steps=500,
  eval_steps=500,
  logging_steps=500,
  learning_rate=1e-4,
  weight_decay=0.005,
  warmup_steps=1000,
  save_total_limit=10,
)
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=asr_data["train"],
    eval_dataset=asr_data["test"],
    tokenizer=processor.feature_extractor,
)
trainer.train()
# trainer.save_checkpoint("speech_recognition_model.pt")

KeyboardInterrupt: ignored

In [ ]:
from transformers import AutoModelForCTC, Wav2Vec2Processor

inference_model = AutoModelForCTC.from_pretrained("/content/drive/MyDrive/wave2vecmodels/checkpoint-1500")



Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [40]:
processor.save_pretrained('/content/drive/MyDrive/dataset/Wave2vec_models/checkpoint-3500')

In [36]:
processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [ ]:
import torchaudio

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [46]:
import torch
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor

# Set the paths for the model and its corresponding processor
model_path = "/content/drive/MyDrive/dataset/Wave2vec_models/checkpoint-3500"
processor_path = "/content/drive/MyDrive/dataset/Wave2vec_models/checkpoint-3500"

# Load the model and processor
model = Wav2Vec2ForCTC.from_pretrained(model_path)
processor = Wav2Vec2Processor.from_pretrained(processor_path)

# (Optional) Set the device for inference (e.g., "cuda" for GPU or "cpu" for CPU)
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

# (Optional) Set the model to evaluation mode

def map_to_result(batch):
  model.eval()
  with torch.no_grad():
    input_values = torch.tensor(batch["input_values"], device="cuda").unsqueeze(0)
    logits = model(input_values).logits
    print(logits.shape)


  pred_ids = torch.argmax(logits, dim=-1)
  batch["pred_str"] = processor.batch_decode(pred_ids)[0]
  batch["text"] = processor.decode(batch["labels"], group_tokens=False)
  
  return batch

results = asr_data["test"].map(map_to_result, remove_columns=asr_data["test"].column_names)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Map:   0%|          | 0/729 [00:00<?, ? examples/s]

torch.Size([1, 324, 32])
torch.Size([1, 533, 32])
torch.Size([1, 316, 32])
torch.Size([1, 707, 32])
torch.Size([1, 440, 32])
torch.Size([1, 344, 32])
torch.Size([1, 437, 32])
torch.Size([1, 599, 32])
torch.Size([1, 902, 32])
torch.Size([1, 509, 32])
torch.Size([1, 728, 32])
torch.Size([1, 725, 32])
torch.Size([1, 425, 32])
torch.Size([1, 860, 32])
torch.Size([1, 695, 32])
torch.Size([1, 422, 32])
torch.Size([1, 572, 32])
torch.Size([1, 455, 32])
torch.Size([1, 407, 32])
torch.Size([1, 446, 32])
torch.Size([1, 683, 32])
torch.Size([1, 371, 32])
torch.Size([1, 698, 32])
torch.Size([1, 527, 32])
torch.Size([1, 506, 32])
torch.Size([1, 620, 32])
torch.Size([1, 467, 32])
torch.Size([1, 548, 32])
torch.Size([1, 245, 32])
torch.Size([1, 485, 32])
torch.Size([1, 365, 32])
torch.Size([1, 398, 32])
torch.Size([1, 740, 32])
torch.Size([1, 383, 32])
torch.Size([1, 575, 32])
torch.Size([1, 920, 32])
torch.Size([1, 509, 32])
torch.Size([1, 1664, 32])
torch.Size([1, 461, 32])
torch.Size([1, 413, 32])

In [ ]:
model.config

In [ ]:
import torchsummary as summary

In [ ]:
summary.summary(model,(1,))

In [47]:
show_random_elements(results)


,pred_str,text
0,in he helped created the siphons with brooks and groning and was responsible for hiring the show's first riting team,in [UNK][UNK][UNK][UNK] he helped create the simpsons with brooks and groening and was responsible for hiring the show's first writing team
1,it uses sattellite[UNK]based technology as opposed to older[UNK]ground[UNK]radar[UNK]based technology to allow air[UNK]traffic controllers to pinpoint aircraft with greater precision and give pilots more accurate information,it uses satellite[UNK]based technology as opposed to older ground[UNK]radar[UNK]based technology to allow air traffic controllers to pinpoint aircraft with greater precision and give pilots more accurate information
2,there are course christian theological explanations for this tradition but it may bwill be a pre[UNK]christian spring and fertility vitual,there are of course christian theological explanations for this tradition but it may well be a pre[UNK]christian spring and fertility ritual
3,its existence is nly is known only because of its effects on the expansion of the universe,its existence is known only because of its effects on the expansion of the universe
4,some people believe that experience o many artificial induced lusly dreams offrom neorlfh can be veryed exausting,some people believe that experiencing many artificially induced lucid dreams often enough can be very exhausting
5,there are infinaute variations possible but this remains what most people mean when they talk of going to disney world,there are infinite variations possible but this remains what most people mean when they talk of going to disney world
6,phases happen because only the side of venus or of the moon facing the sun is lighted the phases of venus supported the theory of capcapernichus that the planets go around the sun,phases happen because only the side of venus or of the moon facing the sun is lighted. the phases of venus supported the theory of copernicus that the planets go around the sun
7,the program started at pm. local time utc,the program started at [UNK][UNK][UNK][UNK] p.m. local time [UNK][UNK].[UNK][UNK] utc
8,the meetere ship was en route to pick up its usual fertalizer cargo and initially officials feared the vessel could spill a load,the [UNK][UNK][UNK][UNK]metre ship was en route to pick up its usual fertiliser cargo and initially officials feared the vessel could spill a load
9,german pastries are quite good and in bavaria are quite rich and varied similar to those of their southern neighbor austria,german pastries are quite good and in bavaria are quite rich and varied similar to those of their southern neighbor austria


In [ ]:
import librosa

In [ ]:
audio_array,sr = librosa.load('/content/Recording.wav',sr=16000) 
# print(sr)
audio_array

/usr/local/lib/python3.8/dist-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


array([-2.0631339e-06,  2.0752846e-05,  3.0953666e-05, ...,
        6.5262680e-04,  3.3885369e-04,  9.2689741e-05], dtype=float32)

In [ ]:
# import torch
# import torchaudio
# from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor

# # Load the fine-tuned model and processor
# model = model
# processor = processor

# def transcribe_audio_file(model,audio_file_path):
#     # Load the audio file and convert to 16,000 Hz sampling rate
#     model.eval()
#     waveform, sample_rate = torchaudio.load(audio_file_path)
#     resampler = torchaudio.transforms.Resample(sample_rate, 16000)
#     waveform = resampler(waveform)

#     # Preprocess the waveform with the fine-tuned processor
#     input_values = processor(waveform, sampling_rate=16000, return_tensors='pt').input_values

#     # Pass the preprocessed waveform through the fine-tuned model
#     with torch.no_grad():
#         logits = model(input_values).logits

#     # Use the processor to decode the logits into text
#     transcript = processor.decode(logits[0])

#     return transcript


In [ ]:
asr_data['train'][0]


{'audio': {'path': 'train/10116315163739194879.wav',
  'array': array([ 0.        ,  0.        ,  0.        , ..., -0.02991241,
         -0.02984595, -0.03363299]),
  'sampling_rate': 16000},
 'transcription': 'massa is due to be out for at least the rest of the 2009 season'}

In [ ]:
asr_data['train'][0]['audio']['path']

'train/10116315163739194879.wav'

In [ ]:
type(tokenizer)

transformers.models.wav2vec2.tokenization_wav2vec2.Wav2Vec2Tokenizer

In [ ]:
!ls /content/drive/MyDrive/wave2vecmodels/checkpoint-1500 -ltrh

In [55]:
# import 
import torch
import librosa
from transformers import Wav2Vec2ForCTC, Wav2Vec2Tokenizer
import torch
import torch.nn.functional as F
max_length = 2048
path='/content/drive/MyDrive/dataset/Wave2vec_models/checkpoint-3500'
processor = processor
# load the tokenizer and model
tokenizer = Wav2Vec2Tokenizer.from_pretrained(path)
model = Wav2Vec2ForCTC.from_pretrained(path)

# load the audio data (use your own wav file here!)
input_audio, sr = librosa.load('/content/00000da4b2da194ac31f6d1466d2ceb4823bdc263efa81004ee89464.wav', sr=16000)

# tokenize
input_values = processor(input_audio, return_tensors="pt", padding="longest").input_values
print(input_values.shape)
# retrieve logits
logits = model(input_values).logits
print(logits.shape)
sequence_length = logits.shape[1]
padding_length = max_length - sequence_length
if padding_length > 0:
    padding = (0,padding_length)
    logits = F.pad(logits, (0, 0, padding[0], padding[1]), "constant", 0)
print(logits.shape)
# take argmax and decode
predicted_ids = torch.argmax(logits, dim=-1)
transcription = tokenizer.batch_decode(predicted_ids)

# print the output
print(transcription)


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'Wav2Vec2CTCTokenizer'. 
The class this function is called from is 'Wav2Vec2Tokenizer'.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


torch.Size([1, 109712])
torch.Size([1, 342, 32])
torch.Size([1, 2048, 32])
['this it is  vany important signal indact asidoside is goldly related to this this.']


In [ ]:
import torch.nn.functional as F
import torch
my_tensor = torch.randn(1, 54, 32)
padding = (0, 10)
padded_tensor = F.pad(my_tensor, (0, 0, padding[0], padding[1]), mode='constant', value=0)


In [ ]:
my_tensor.shape

torch.Size([1, 54, 32])

In [ ]:
padded_tensor.shape

torch.Size([1, 64, 32])

In [ ]:
padded_tensor

tensor([[[ 2.2457, -1.7760,  0.6908,  ..., -0.7843, -1.2532,  0.4244],
         [ 1.2654, -0.7177, -0.3560,  ...,  0.5529,  1.0246,  1.1805],
         [ 0.3647, -0.8909, -0.4678,  ...,  1.1897, -1.8964,  0.4678],
         ...,
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]]])

In [ ]:
transcribe_audio_file(model,'/content/Recording.wav')

RuntimeError: ignored